In [54]:
import pandas as pd
import dash
from dash import dcc, html, Input, Output
import plotly.express as px

In [55]:
url = "https://en.wikipedia.org/wiki/List_of_FIFA_World_Cup_finals"
master = pd.read_html(url)
df = pd.DataFrame()

# find the first table that contains necessary columns
for item in master:
    if "Years won" in item.columns and "Team" in item.columns and "Years runners-up" in item.columns:
        df = item
        break

In [59]:
#create list of (Year, Winner) tuples
winners = []
for _, row in df.iterrows():
    if pd.notna(row["Years won"]):
        years = [year.strip() for year in str(row["Years won"]).split(',')]
        for year in years:
            if year != '—':
                winners.append((year, row["Team"]))

# create list of (Year, Runner-up) tuples
runners_up = []
for _, row in df.iterrows():
    if pd.notna(row["Years runners-up"]):
        years = [year.strip() for year in str(row["Years runners-up"]).split(',')]
        for year in years:
            if year != '—':
                runners_up.append((year, row["Team"]))

# convert to DataFrames and merge
winners = pd.DataFrame(winners, columns=["Year", "Winner"])
runners_up = pd.DataFrame(runners_up, columns=["Year", "Runner-up"])
world_cup = pd.merge(winners, runners_up, on="Year", how="outer")

world_cup

,Year,Winner,Runner-up
0,1930,Uruguay,Argentina
1,1934,Italy,Czechoslovakia
2,1938,Italy,Hungary
3,1950,Uruguay,Brazil
4,1954,Germany,Hungary
5,1958,Brazil,Sweden
6,1962,Brazil,Czechoslovakia
7,1966,England,Germany
8,1970,Brazil,Italy
9,1974,Germany,Netherlands


In [57]:
app = dash.Dash(__name__)
server = app.server 

# Count total wins by country
wins_count = world_cup['Winner'].value_counts().reset_index()
wins_count.columns = ['Country', 'Wins']

app.layout = html.Div([
    html.H1("FIFA World Cup Winners and Runner Ups", style={'textAlign': 'center'}),

    html.H2("Select a country:"),
    dcc.Dropdown(
        id='country-dropdown',
        options=[{'label': c, 'value': c} for c in sorted(world_cup['Winner'].unique())],
        value='Brazil'
    ),
    html.Div(id='country-win-output'),

    html.H2("Select a year to see the winner and runner up:"),
    dcc.Dropdown(
        id='year-dropdown',
        options=[{'label': y, 'value': y} for y in sorted(world_cup['Year'].unique())],
        value=world_cup['Year'].min()
    ),
    html.Div(id='year-result-output'),

    html.H2("World Cup Wins by Country"),
    dcc.Graph(id='choropleth-map')
])


@app.callback(
    Output('country-win-output', 'children'),
    Input('country-dropdown', 'value')
)
def update_country_wins(country):
    wins = world_cup[world_cup['Winner'] == country].shape[0]
    string = f"{country} has won the World Cup {wins} time(s)"
    return string

@app.callback(
    Output('year-result-output', 'children'),
    Input('year-dropdown', 'value')
)
def update_year_result(year):
    row = world_cup[world_cup['Year'] == year].iloc[0]
    string = f"In {year}, {row['Winner']} won the World Cup and the runner-up was {row['Runner-up']}."
    return string


@app.callback(
    Output('choropleth-map', 'figure'),
    Input('country-dropdown', 'value')  
)
def update_map(_):
    fig = px.choropleth(
        wins_count,
        locations="Country",
        locationmode="country names",
        color="Wins"
    )
    return fig


if __name__ == '__main__':
    app.run(debug=True)
